<a href="https://colab.research.google.com/github/AArna1211/Traffic-prediction/blob/main/TP_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

data = pd.read_csv('/content/Traffic.csv')

# Convert Time & Date columns to datetime format
data["Timestamp"] = pd.to_datetime(data["Time"] + " " + data["Date"].astype(str))
data = data.sort_values(by="Timestamp")

# Encode categorical "Traffic Situation"
label_encoder = LabelEncoder()
data["Traffic Situation Encoded"] = label_encoder.fit_transform(data["Traffic Situation"])

# Select features for LSTM
features = ["CarCount", "BikeCount", "BusCount", "TruckCount", "Total"]
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[features])

# Create time-series sequences
def create_sequences(data, labels, seq_length=2):  # Adjusted for small dataset
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(labels[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 2
X, y = create_sequences(data_scaled, data["Traffic Situation Encoded"].values, seq_length)
X = X.reshape(X.shape[0], X.shape[1], len(features))  # Reshape for LSTM

# Split into train and test sets
split = int(0.8 * len(X))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

# Build LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(seq_length, len(features))),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
epochs = 20
batch_size = 1  # Adjusted for small dataset
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

# Predict on test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Decode labels
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
y_test_labels = label_encoder.inverse_transform(y_test)

# Evaluate model
accuracy = np.mean(y_pred_classes == y_test)
print(f"Test Accuracy: {accuracy:.4f}")


<ipython-input-1-c6e80437b0e7>:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Timestamp"] = pd.to_datetime(data["Time"] + " " + data["Date"].astype(str))
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
2379/2379 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - accuracy: 0.6269 - loss: 0.9851 - val_accuracy: 0.7378 - val_loss: 0.7753
Epoch 2/20
2379/2379 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7262 - loss: 0.7805 - val_accuracy: 0.7261 - val_loss: 0.7771
Epoch 3/20
2379/2379 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7104 - loss: 0.7941 - val_accuracy: 0.7361 - val_loss: 0.7560
Epoch 4/20
2379/2379 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.7377 - loss: 0.7431 - val_accuracy: 0.7429 - val_loss: 0.7469
Epoch 5/20
2379/2379 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7294 - loss: 0.7594 - val_accuracy: 0.7378 - val_loss: 0.7358
Epoch 6/20
2379/2379 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7320 - loss: 0.7503 - val_accuracy: 0.7462 - val_loss: 0.7448
Epoch 7/20
2379/2379 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.7421 - loss: 0.7297 - val_accuracy: 0.7395 - val_loss: 0.7597
Epoch 8/20
2379/2379 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.7193 - loss: 0.